In [1]:
import pinecone
import os
import boto3

client=boto3.client('textract',region_name='YOUR REGION',aws_access_key_id="YOUR ACCESS KEY",
                    aws_secret_access_key="YOUR SECRET KEY")

pinecone.init(api_key = "YOUR API KEY",environment = "YOUR ENVIRONMENT")

In [3]:
pinecone.create_index("INDEX NAME", dimension=384, source_collection="INDEX NAME")

In [2]:
def get_response(image):
    response=client.detect_document_text(
        Document={'Bytes':image}
    )
    text=""
    for item in response['Blocks']:
        if item['BlockType']=='LINE':
            print(item['Text'])
            text += f" {item['Text']}"
    
    return text

In [ ]:
for dir in os.listdir("docs-sm"):
    for items in os.listdir(os.path.join("docs-sm",dir)):
        image_path = os.path.join(os.path.join("docs-sm",dir,items))
        with open(image_path,'rb') as image:    
            processed_image = bytearray(image.read())
        print((image_path,dir))
        ocr_response = get_response(processed_image)
        

In [ ]:
from tqdm.auto import tqdm
ocr_items = []
for dir in tqdm(os.listdir("docs-sm")):
    for idx,items in tqdm(enumerate(os.listdir(os.path.join("docs-sm",dir)))):
        image_path = os.path.join(os.path.join("docs-sm",dir,items))
        ocr_response = get_response(image_path)
        ocr_items.append((dir,ocr_response))
        if idx == 40:
            break
        
        

In [5]:
ocr_texts = []
for item in ocr_items:
    for dic in item[1].items():
        ocr_texts.append((item[0],dic[1]))

In [6]:
import pandas as pd

df = pd.DataFrame(ocr_texts,columns=['doc_type','ocr_text'])

In [7]:
df

,doc_type,ocr_text
0,advertisement,FREE SAMPLE \n. \nRALEIGH \nRALEIGH \nLIGHTS \...
1,advertisement,STORS COUPON \nALDA $ 100 OFF \n- : // h \nJot...
2,advertisement,***** \n253 \nrehiy \nVintage pleasures \nWhen...
3,advertisement,I 57 find % of clean Winston tasting smokers F...
4,advertisement,Latest US.Government figures show \nPALL MALL ...
...,...,...
636,specification,Section Q.E. Code \nPHILIP MORRIS 1 \n( PH ) S...
637,specification,U.S. DEPARTMENT OF LABOR Form Approved \nOccup...
638,specification,لمتحكما wcowards Lauce nici Company mses SOS #...
639,specification,CIGARETTE SPECIFICATION \nLOUISVILLE EFFECTIVE...


In [8]:
df.to_csv("Upsertable Docs.csv",index=None)

In [9]:
len(ocr_texts)

641

In [12]:
from sentence_transformers import SentenceTransformer

model_name = "all-minilm-l6-v2"
model = SentenceTransformer(model_name)

In [13]:
def get_embeddings(text):
    return model.encode(text).tolist()

In [7]:
id = index.describe_index_stats()['total_vector_count']

In [8]:
id

0

In [10]:
import pandas as  pd

df = pd.read_csv("Upsertable Docs.csv")

In [14]:
from tqdm.auto import tqdm
docs_list = []
id
for i, row in tqdm(df.iterrows()):
    vector_id =  str(id+i)
    docs_list.append((
        vector_id, get_embeddings(row['ocr_text']),{
            'ocr_text':row['ocr_text'],
            'doctype':row['doc_type']
        })
    )
    index.upsert(vectors=docs_list)

0it [00:00, ?it/s]

In [21]:
query = "Hello, I Hope you are doing well, Please Buy our Lottery tickets for 100$"
xq = model.encode([query]).tolist()
result = index.query(xq,top_k=1,include_metadata=True)
print(result)

{'matches': [{'id': '16',
              'metadata': {'doctype': 'advertisement',
                           'ocr_text': "You don't have a \n"
                                       'to be elected President \n'
                                       'to live like one \n'
                                       'Win $ 16,000 in the Kent & True '
                                       'Presidential Sweepstakes ! \n'
                                       'Or any one of 5,125 other prizes ! \n'
                                       'C KENT TRUE \n'
                                       'AN AZE \n'
                                       'KENT 6N 321 \n'
                                       '10 \n'
                                       'TRUE . \n'
                                       'First Prize \n'
                                       "The each President's year for 4 weekly "
                                       'years- salary $ 16,000 , $ 4,000 in '
                             